# Message Passing
## Built-in Functions and Message Passing APIs
`message function` takes a single argument `edges`, and this has three members: `src`,`dst` and `data` to access features of source nodes, destination nodes and edges.

`reduce function` takes a single argument `nodes`, and uses a member called `mailbox` to access the messages reveived for the nodes in the batch.

`update function` takes a single argument `nodes` to aggregate messages from various nodes.

The following is an example that can `add` the `hu` feature from src nodes and `hv` feature from dst nodes and save the result on the edge at `he` field.

In [2]:
def message_func(edges):
    return {'he':edges.src['hu'] + edges.dst['hv']}

This following equivalent to the Reduce UDF that sums up the message `m`.

In [4]:
import torch 
def reduce_func(nodes):
    return {'h': torch.sum(nodes.mailbox['m'],dim=1)}

This following uses a function called `apply_edges` to apply certain feature with the parameter to gain updating.

In [5]:
import dgl.function as fn 
# graph.apply_edges(fn.u_add_v('el', 'er', 'e'))

The following is an example of update all information on a graph.

It multiplies the src node features `ft` and edge features `a` and sum up the messages `m` to update the node features `ft`, and then multiplies 2 to get the result `final_ft`. After this call, DGL will clean the intermediate messages `m`. The formula can be described as follows:

$$final\_ft_i = 2 * \sum\limits_{j \in graph(i)}(ft_j * a_{ji})$$

In [ ]:
def update_all_example(graph):
    graph.update_all(fn.u_mul_e('ft','a','m'),
                     fn.sum('m','ft'))
    final_ft = graph.ndata['ft'] * 2
    return final_ft

## Message Passing on Heterogeneous Graph
Haterogeneous graphs are graphs that contain different types of nodes and edges. The different types tend to have different types of attributes that are designed to capture the characteristics of each node and edge type.

The following contains how a heterograph can be used for message passing.

In [ ]:
import dgl.function as fn

for c_etype in G.canonical_etypes:
    srctype, etype, dsttype = c_etype
    Wh = self.weight[etype](feat_dict[srctype])
    # Save it in graph for message passing
    G.nodes[srctype].data['Wh_%s' % etype] = Wh
    # Specify per-relation message passing functions: (message_func, reduce_func).
    # Note that the results are saved to the same destination feature 'h', which
    # hints the type wise reducer for aggregation.
    funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
# Trigger message passing of multiple types.
G.multi_update_all(funcs, 'sum')
# return the updated node feature dictionary
return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}